In [7]:
from google_play_scraper import Sort, reviews, app
import pandas as pd
import pickle

def fraud_review(app_id):
  scd=[]
  app_packages=[app_id]
  for i in app_packages:
    result = app(i,
      lang='en', # defaults to 'en'
      country='in') # defaults to 'us'
    scd.append(result)
  df_master=pd.DataFrame(scd)
  df = df_master
  rvsum=0
  for j in range(0,len(df)):
    rvsum+=df['reviews'].loc[j]
  app_reviews = []

  for ap in (app_packages):
    for score in list(range(1, 6)):
      for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
        rvs, _ = reviews(
          ap,
          lang='en',
          country='in',
          sort=sort_order,
          count= 200 if score == 3 else 100,
          filter_score_with=score
        )
        for r in rvs:
          r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
          r['appId'] = ap
          #print(r)
        app_reviews.extend(rvs)
        
  df_app=pd.DataFrame(app_reviews)    
  vec = pickle.load(open("Fraud_Review_1.pkl","rb"))
  con = vec.transform(df_app['content'])
  sca = pickle.load(open("Fraud_Review_2.pkl","rb")) ##Scaler Load
  con = sca.transform(con)
  lrm = pickle.load(open("Fraud_Review_3.pkl","rb"))  ##Lr Load
  pred= lrm.predict(con)

  avg_score=(sum(pred))/len(pred)  
  
  return avg_score

In [ ]:
if __name__ == "__main__":
    app_id = 'com.kotak.kst'
    print(fraud_review(app_id))